# 2D to 3D LSTM

This is the first approach to try to estimate 3D points coordinates from 2D keypoints extracted with Openpose. Here I will build a simple LSTM to perform the task over the Panoptic Studio dataset.

First, let's import all the necessary libraries.

In [35]:
# Pytorch utilities
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from scipy.spatial.transform import Rotation as R
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Plotting utilities
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import matplotlib.animation as animation
from torch.utils.tensorboard import SummaryWriter
from timeit import default_timer as timer
import pyprind

# Directory and file utilities
from os import listdir
from os.path import isfile, isdir, join
import json

## Notebook parameters

In [36]:
modes = { 0:'one video', 1:'one signer', 2:'all signers'}
MODE =  modes[1]
video = 3

## Data acquisition
Now I will define some functions in order to parse and organise the data, and later convert it to pytorch tensors.

The data is structured as follows: in the dataset directory there are several folders, each folder corresponds to a recording; each of these folders contains a folder with the audio, folders with face, hands and body keypoints estimations for each frame, and a folder with the video recorded from different views.

In this first approach I will be using the keypoints estimations. Every keypoint folder (face, hands or body) is organized the same way: it contains a json per frame of the video, which includes the 3D keypoints estimation.

The function `get_keypoints` will go through each folder in the dataset directory and retrieve the face keypoints, the hands keypoints and the body keypoints. It will separate them into input (2D coordinates per joint per frame) and grountruth (third coordinate to estimate for each input 2D keypoint). 
The input will be of shape $([n videos, seq len, input size])$, where *seq_len* = number of frames, and *input_size* = face + hands + body keypoints, that is (70+(21+21)+26)x2 -multiplied by 2 because there are x and y coordinates-. The groundtruth (label) data will be of the same shape, except that the last dimension size will not be multiplied by 2 (there's only one coordinate to estimate).  

In [3]:
def get_keypoints(data_path):
    dataset = []
    groundtruth = []
    # Look over just the folders inside the directory
    just_folders = filter(lambda x: isdir(join(data_path, x)), listdir(data_path))
    for p in list(map(lambda x: join(data_path, x), just_folders)): 
        # Gets 2 list of n_frames lists, one for the 2D coordinates and one for the third coordinate.
        # Each list of the n_frames lists contains, either the (x and y) or the z of each keypoint for the face(first line), hands(second), body(third).
        # e.g. the first line will result in [[x1,y1,x2,y2...x70,y70]sub1...[x1,y1...x70,y70]subN], [[z1,z2...z70]sub1...[z1..z70]subN]
        # Actually, as there will be two of each list above because there are two people en each video.
        pose_2d, pose_3d = get_body(p)
        
        # Concatenates the coordinates for the face, hands and body on the last dimension, for each person.
        vid_input_p1, vid_input_p2 = pose_2d
        vid_labels_p1, vid_labels_p2 = pose_3d
        
        dataset.append(vid_input_p1)
        dataset.append(vid_input_p2)
        groundtruth.append(vid_labels_p1)
        groundtruth.append(vid_labels_p2)
        print(f'Completed folder {p}')
    return dataset, groundtruth

The body json is organised a bit differently, inside each person object contains the *joints26* field with a list of 3D coordinates. But this list is structured as follows: *[x1,y1,z1,acc1,x2,y2,z2,acc2...]*.

In [4]:
def get_body(path):
    body_2D_seq = ([], [])
    body_3D_seq = ([], [])
    paths = map(lambda x: join(path, 'hdPose3d_stage1_op25', x), sorted(listdir(join(path, 'hdPose3d_stage1_op25'))))
    files = list(filter(lambda x: isfile(x), paths))
    for f in files:
        with open(f, 'r') as j:
            json_array = json.load(j)
            i = 0
            for person in json_array['bodies']:
                if person['id'] != -1:
                    x = person['joints26'][::4]
                    y = person['joints26'][1::4]
                    two_coord = [l[item] for item in range(len(x)) for l in [x,y]]
                    third_coord = person['joints26'][2::4]
                    body_2D_seq[person['id']].append(two_coord)
                    body_3D_seq[person['id']].append(third_coord)
                    i += 1
                    pid = person['id']
            if i<2:
                body_2D_seq[1-pid].append(body_2D_seq[1-pid][-1])
                body_3D_seq[1-pid].append(body_3D_seq[1-pid][-1])
    print('Body completed.')
    return body_2D_seq, body_3D_seq

In [5]:
data_path = '../../../data/DB keypoints'
dataset, groundtruth = get_keypoints(data_path)

Body completed.
Completed folder ../../../data/DB keypoints/190419_asl2
Body completed.
Completed folder ../../../data/DB keypoints/190419_asl4
Body completed.
Completed folder ../../../data/DB keypoints/190419_asl5
Body completed.
Completed folder ../../../data/DB keypoints/190425_asl1
Body completed.
Completed folder ../../../data/DB keypoints/190425_asl2
Body completed.
Completed folder ../../../data/DB keypoints/190425_asl3
Body completed.
Completed folder ../../../data/DB keypoints/190425_asl5
Body completed.
Completed folder ../../../data/DB keypoints/190425_asl7
Body completed.
Completed folder ../../../data/DB keypoints/190425_asl9
Body completed.
Completed folder ../../../data/DB keypoints/190425_asl91
Body completed.
Completed folder ../../../data/DB keypoints/190611_asl1
Body completed.
Completed folder ../../../data/DB keypoints/190611_asl2
Body completed.
Completed folder ../../../data/DB keypoints/190611_asl3
Body completed.
Completed folder ../../../data/DB keypoints/190

In [6]:
def padding_seq(dataset):
    max_seq = max([len(x) for x in dataset])
    seq_lengths = []
    for seq in dataset:
        seq_lengths.append(len(seq))
        for i in range(max_seq-len(seq)):
            seq.append([np.nan for j in range(len(seq[0]))])
        
    return max_seq, seq_lengths

max_seq, seq_lengths = padding_seq(dataset)
_, _ = padding_seq(groundtruth)

In [7]:
# From python lists to numpy ndarray.
dataset = np.asarray(dataset)
groundtruth = np.asarray(groundtruth)
lengths = np.asarray(seq_lengths)
print(dataset.shape, groundtruth.shape, lengths.shape)

(44, 8752, 52) (44, 8752, 26) (44,)


In [8]:
np.save('../../pickles/body_data.npy', dataset)
np.save('../../pickles/body_ground.npy', groundtruth)

In [9]:
np.save('../../pickles/body_lengths.npy', lengths)

## Dataset structuring
Now let's convert the lists obtained to Pytorch tensors and organise them in train, validation and test datasets. 
First, I will define a padding function in order to make all the sequences of video frames the same length, so I can train the LSTM in batches. 

#### Load from pickle
Load keypoints from pre-saved pickle files instead of directly reading the jsons, can be found in below cells

Once they are numpy ndarrays I save the keypoints into pickle files for faster loading in later executions

In [65]:
dataset = np.load('../../pickles/body_data.npy', allow_pickle=True)
groundtruth = np.load('../../pickles/body_ground.npy', allow_pickle=True)
lengths = np.load('../../pickles/body_lengths.npy', allow_pickle=True)
max_seq = dataset.shape[1]
print(max_seq)

8752


In [66]:
print(lengths)

[8272 8272 8272 8272 8271 8271 8751 8751 5952 5952 7611 7611 6953 6953
 6012 6012 8574 8574 5176 5176 8045 8045 7836 7836 7326 7326 7141 7141
 8751 8751 5155 5155 8751 8751 8752 8752 8751 8751 8751 8751 8751 8751
 7205 7205]


In [67]:
if MODE == 'one video':
    dataset, groundtruth = dataset[video], groundtruth[video]
    chunks_d = np.split(dataset, 547, axis=0)[:lengths[video]//16]
    chunks_g = np.split(groundtruth, 547, axis=0)[:lengths[video]//16]
    chunks_d = tuple(np.expand_dims(c, axis=0) for c in chunks_d)
    chunks_g = tuple(np.expand_dims(c, axis=0) for c in chunks_g)
    
    dataset = np.concatenate(chunks_d, axis=0)
    groundtruth = np.concatenate(chunks_g, axis=0)
    lengths = np.asarray([16 for i in range(lengths[video]//16)])
    print(lengths.shape)

elif MODE == 'one signer':
    dataset, groundtruth, lengths = dataset[20::2, :-2], groundtruth[20::2, :-2], lengths[20::2]
    print(lengths)
    lengths = np.asarray([min(8750, x) for x in lengths])
    print(lengths)
    chunks_d, chunks_g = np.split(dataset, 35, axis=1), np.split(groundtruth, 35, axis=1)
    
    dataset, groundtruth = np.concatenate(chunks_d, axis=0), np.concatenate(chunks_g, axis=0)
    lengths = np.concatenate(tuple([250 if j*250<=lengths[i] 
                                    else (lengths[i]%250 if (j-1)*250<lengths[i] 
                                          else 0) for i in range(12)] for j in range(1,36)), axis=0)
    print(dataset.shape, groundtruth.shape, lengths.shape)
    
elif MODE == 'all signers':
    dataset, groundtruth = dataset[:, :-2], groundtruth[:, :-2]
    lengths = np.asarray([min(8750, x) for x in lengths])
    chunks_d, chunks_g = np.split(dataset, 25, axis=1), np.split(groundtruth, 25, axis=1)
    
    dataset, groundtruth = np.concatenate(chunks_d, axis=0), np.concatenate(chunks_g, axis=0)
    lengths = np.concatenate(tuple([350 if j*350<=lengths[i]
                                    else (lengths[i]%350 if (j-1)*350<lengths[i] 
                                          else 0) for i in range(44)] for j in range(1,26)), axis=0)
    print(dataset.shape, groundtruth.shape, lengths.shape)

[8045 7836 7326 7141 8751 5155 8751 8752 8751 8751 8751 7205]
[8045 7836 7326 7141 8750 5155 8750 8750 8750 8750 8750 7205]
(420, 250, 52) (420, 250, 26) (420,)


In [68]:
# Clean all NaN videos
dataset = np.delete(dataset, np.where(lengths==0), axis=0)
groundtruth = np.delete(groundtruth, np.where(lengths==0), axis=0)
lengths = np.delete(lengths, np.where(lengths==0), axis=0)

print(dataset.shape, groundtruth.shape, lengths.shape)

(384, 250, 52) (384, 250, 26) (384,)


In [69]:
def align(tensor, coordinates=1):
    for n_vid in range(tensor.shape[0]):
        max_value = [np.nanmax(tensor[n_vid, :,i::coordinates]) for i in range(coordinates)]
        min_value = [np.nanmin(tensor[n_vid, :,i::coordinates]) for i in range(coordinates)]
        center = [(max_value[i]+min_value[i])/2 for i in range(coordinates)]
        for j in range(coordinates):
            subtensor = tensor[n_vid, :, j::coordinates]
            subtensor[:] = np.subtract(subtensor, center[j])

align(dataset,2)
align(groundtruth)
print(dataset.shape, groundtruth.shape, lengths.shape)

(384, 250, 52) (384, 250, 26) (384,)


In [70]:
r = R.from_euler('y', 110, degrees=True)
shapes = dataset.shape
dataset = dataset.reshape(-1, 2)
groundtruth = groundtruth.reshape(-1,1)
xyz = np.concatenate((dataset, groundtruth), axis=1)
xyz = r.apply(xyz)
dataset, groundtruth = xyz[:, :2].reshape(shapes), xyz[:,2].reshape(shapes[0], shapes[1],26)

print(dataset.shape, groundtruth.shape)

(384, 250, 52) (384, 250, 26)


In [71]:
def norm_uniform(tensor, coordinates=1, factor=None):
    scale = []
    mean_ranges = []
    for n_vid in range(tensor.shape[0]):
        coord_scale = []
        max_value = [np.nanmax(tensor[n_vid, :,i::coordinates]) for i in range(coordinates)]
        min_value = [np.nanmin(tensor[n_vid, :,i::coordinates]) for i in range(coordinates)]
        center = [(max_value[i]+min_value[i])/2 for i in range(coordinates)]
        ranges = np.ndarray((tensor.shape[1],coordinates))
        for n_frame in range(tensor.shape[1]):
            rang = [np.nanmax(tensor[n_vid, n_frame,i::coordinates])-np.nanmin(tensor[n_vid, n_frame,i::coordinates]) for i in range(coordinates)]
            ranges[n_frame] = np.asarray(rang)
        mean_range = [np.nanmean(ranges[:,i]) for i in range(coordinates)]
        for j in range(coordinates):
            subtensor = tensor[n_vid, :, j::coordinates]
            subtensor[:] = np.subtract(subtensor, center[j])
            if factor is not None:
                subtensor[:] = np.divide(subtensor, factor[n_vid])
            else:
                subtensor[:] = np.divide(subtensor, max_value[j]-center[j])
            coord_scale.append((max_value[j]-center[j] if factor is None else factor[n_vid]))
        scale.append(coord_scale)
        mean_ranges.append(mean_range)
    return mean_ranges
input_scale = np.asarray(norm_uniform(dataset,2)).squeeze()
print(input_scale.shape)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: All-NaN slice encountered
  # This is added back by InteractiveShellApp.init_path()


(384, 2)


In [72]:
def normalize(tensor, coordinates=1, std=None):
    moments = []
    std_centroids = []
    for n_vid in range(tensor.shape[0]):
        coord_moments = []
        mean_value = [np.nanmean(tensor[n_vid, :,i::coordinates]) for i in range(coordinates)]
        std_value = [np.nanstd(tensor[n_vid, :,i::coordinates]) for i in range(coordinates)]
        centroids = np.ndarray((tensor.shape[1],coordinates))
        for n_frame in range(tensor.shape[1]):
            centroid = [np.nanmean(tensor[n_vid, n_frame, i::coordinates]) for i in range(coordinates)]
            centroids[n_frame] = np.asarray(centroid)
        std_centroid = [np.nanstd(centroids[:,i]) for i in range(coordinates)]
        if std is not None:
            std_value = [std[n_vid]]
        for j in range(coordinates):
            subtensor = tensor[:, :, j::coordinates]
            subtensor[:] = np.subtract(subtensor, mean_value[j])
            subtensor[:] = np.divide(subtensor, std_value[j])
            coord_moments.append((mean_value[j], std_value[j]))
        moments.append(coord_moments)
        std_centroids.append(std_centroid)
    return moments, std_centroids

moments, std_centroids = normalize(dataset, 2)
mom_x, mom_y = np.asarray(moments)[:, 0], np.asarray(moments)[:, 1]
std_centroids = np.asarray(std_centroids)
print(mom_x.shape, mom_y.shape)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


(384, 2) (384, 2)


In [73]:
# Randomly shuffle videos
permutation = np.random.permutation(dataset.shape[0])
dataset, groundtruth, lengths = dataset[permutation], groundtruth[permutation], lengths[permutation]
input_scale, mom_x, mom_y = input_scale[permutation], mom_x[permutation], mom_y[permutation]
std_centroids = std_centroids[permutation]

print(dataset.shape, groundtruth.shape, lengths.shape)
print(input_scale.shape, mom_x.shape, mom_y.shape)

(384, 250, 52) (384, 250, 26) (384,)
(384, 2) (384, 2) (384, 2)


In [74]:
l1, l2 = len(dataset), len(groundtruth)
p1, p2 = 0.8, 0.9
# Split in train, validation and test
training_kp, val_kp, test_kp = dataset[:round(p1*l1)], dataset[round(p1*l1):round(p2*l1)], dataset[round(p2*l1):]
training_lbl, val_lbl, test_lbl = groundtruth[:round(p1*l2)], groundtruth[round(p1*l2):round(p2*l2)], groundtruth[round(p2*l2):]
training_lengths, val_lengths, test_lengths = lengths[:round(p1*l1)], lengths[round(p1*l1):round(p2*l1)], lengths[round(p2*l1):]
training_inpscale, val_inpscale, test_inpscale = input_scale[:round(p1*l1)], input_scale[round(p1*l1):round(p2*l1)], input_scale[round(p2*l1):]
training_mom_x, val_mom_x, test_mom_x = mom_x[:round(p1*l1)], mom_x[round(p1*l1):round(p2*l1)], mom_x[round(p2*l1):]
training_mom_y, val_mom_y, test_mom_y = mom_y[:round(p1*l1)], mom_y[round(p1*l1):round(p2*l1)], mom_y[round(p2*l1):]

training_std_centroids, val_std_centroids, test_std_centroids = std_centroids[:round(p1*l1)], std_centroids[round(p1*l1):round(p2*l1)], std_centroids[round(p2*l1):]

print(training_kp.shape, val_kp.shape, test_kp.shape)
print(training_lbl.shape, val_lbl.shape, test_lbl.shape)
print(training_lengths.shape, val_lengths.shape, test_lengths.shape)

(307, 250, 52) (39, 250, 52) (38, 250, 52)
(307, 250, 26) (39, 250, 26) (38, 250, 26)
(307,) (39,) (38,)


In [75]:
training_outscale = np.asarray(norm_uniform(training_lbl)).squeeze()
val_outscale_t = np.asarray(norm_uniform(val_lbl)).squeeze()
test_outscale_t = np.asarray(norm_uniform(test_lbl)).squeeze()
print(training_outscale.shape, val_outscale_t.shape)

training_mom_z = np.asarray(normalize(training_lbl)[0])[:, 0, 1]
val_mom_z_t = np.asarray(normalize(val_lbl)[0])[:, 0, 1]
test_mom_z_t = np.asarray(normalize(test_lbl)[0])[:, 0, 1]
print(training_mom_z.shape, val_mom_z_t.shape)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: All-NaN slice encountered
  # This is added back by InteractiveShellApp.init_path()


(307,) (39,)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


(307,) (39,)


In [76]:
alpha = LinearRegression(fit_intercept=True)#make_pipeline(PolynomialFeatures(9), LinearRegression(fit_intercept=True))
alpha.fit(training_inpscale, training_outscale)
val_outscale = alpha.predict(val_inpscale)
test_outscale = alpha.predict(test_inpscale)
print(val_outscale.shape, test_outscale.shape)

sigma = LinearRegression(fit_intercept=True)#make_pipeline(PolynomialFeatures(9), LinearRegression(fit_intercept=True))
train_X = np.concatenate((training_mom_x[:,1, np.newaxis], training_mom_y[:,1, np.newaxis]), axis=1)
sigma.fit(train_X, training_mom_z)
val_X = np.concatenate((val_mom_x[:,1,np.newaxis], val_mom_y[:,1,np.newaxis]), axis=1)
test_X = np.concatenate((test_mom_x[:,1,np.newaxis], test_mom_y[:,1,np.newaxis]), axis=1)
val_mom_z = sigma.predict(val_X)
test_mom_z = sigma.predict(test_X)
print(val_mom_z.shape, test_mom_z.shape)

(39,) (38,)
(39,) (38,)


In [15]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(training_inpscale[:,0], training_inpscale[:,1], training_outscale)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(training_std_centroids[:,0], training_std_centroids[:,1], training_mom_z)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
plt.figure()
plt.scatter(training_std_centroids[:,0], training_mom_z)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
plt.figure()
plt.scatter(training_std_centroids[:,1], training_mom_z)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [77]:
from sklearn.metrics import mean_squared_error
print(val_outscale_t, test_outscale_t)
print(val_mom_z_t, test_mom_z_t)
print(mean_squared_error(val_outscale, val_outscale_t), mean_squared_error(test_outscale, test_outscale_t))
print(mean_squared_error(val_mom_z, val_mom_z_t), mean_squared_error(test_mom_z, test_mom_z_t))

[33.16890364 37.52600716 35.68102743 29.10770817 46.45064291 31.87390527
 33.49964475 24.68848576 40.93718594 35.72862974 35.82597025 35.34950068
 36.61202844 34.1738308  31.8796568  28.3604937  30.93998545 42.29560358
 39.74829792 29.7574219  32.14683906 35.79208135 32.11913589 43.17918223
 38.65516213 35.64807295 28.96494507 36.14946187 36.34473658 39.63451905
 39.28940898 36.71739752 39.10168948 33.65543512 43.19246471 33.12356447
 25.3362475  35.04416996 39.6292588 ] [35.06653597 38.03199294 31.09010684 33.90641443 40.42159184 33.17555203
 25.26472814 43.46014919 32.97650161 36.12971844 37.11721015 37.96395941
 32.3162539  26.83848327 38.88588793 31.32611547 28.84628992 33.58679553
 37.15287509 36.22073181 26.36969244 37.98296445 43.1265289  37.42641604
 36.5980991  34.26758739 31.61812701 32.47128821 36.22420301 38.28726241
 32.79081379 37.99922584 37.61571807 26.99234537 37.80717421 33.90795695
 31.99203956 31.90600776]
[0.40746163 0.91672865 1.03880107 1.22839301 0.98453862 0.86

In [78]:
val_outscale = np.concatenate((val_outscale[:,np.newaxis], val_outscale_t[:,np.newaxis]), axis=1)
test_outscale = np.concatenate((test_outscale[:,np.newaxis], test_outscale_t[:,np.newaxis]), axis=1)

val_mom_z = np.concatenate((val_mom_z[:,np.newaxis], val_mom_z_t[:,np.newaxis]), axis=1)
test_mom_z = np.concatenate((test_mom_z[:,np.newaxis], test_mom_z_t[:,np.newaxis]), axis=1)
print(val_outscale.shape, test_mom_z.shape)

(39, 2) (38, 2)


In [79]:
# From python lists to pytorch tensors.
training_kp, val_kp, test_kp = torch.tensor(np.nan_to_num(training_kp), dtype=torch.float32), torch.tensor(np.nan_to_num(val_kp), dtype=torch.float32), torch.tensor(np.nan_to_num(test_kp), dtype=torch.float32)
training_lbl, val_lbl, test_lbl = torch.tensor(np.nan_to_num(training_lbl), dtype=torch.float32), torch.tensor(np.nan_to_num(val_lbl), dtype=torch.float32), torch.tensor(np.nan_to_num(test_lbl), dtype=torch.float32)
training_lengths, val_lengths, test_lengths = torch.tensor(np.nan_to_num(training_lengths), dtype=torch.float32), torch.tensor(np.nan_to_num(val_lengths), dtype=torch.float32), torch.tensor(np.nan_to_num(test_lengths), dtype=torch.float32)
training_inpscale, val_inpscale, test_inpscale = torch.tensor(np.nan_to_num(training_inpscale), dtype=torch.float32), torch.tensor(np.nan_to_num(val_inpscale), dtype=torch.float32), torch.tensor(np.nan_to_num(test_inpscale), dtype=torch.float32)
training_outscale, val_outscale, test_outscale = torch.tensor(np.nan_to_num(training_outscale), dtype=torch.float32), torch.tensor(np.nan_to_num(val_outscale), dtype=torch.float32), torch.tensor(np.nan_to_num(test_outscale), dtype=torch.float32)
training_mom_x, val_mom_x, test_mom_x = torch.tensor(np.nan_to_num(training_mom_x), dtype=torch.float32), torch.tensor(np.nan_to_num(val_mom_x), dtype=torch.float32), torch.tensor(np.nan_to_num(test_mom_x), dtype=torch.float32)
training_mom_y, val_mom_y, test_mom_y = torch.tensor(np.nan_to_num(training_mom_y), dtype=torch.float32), torch.tensor(np.nan_to_num(val_mom_y), dtype=torch.float32), torch.tensor(np.nan_to_num(test_mom_y), dtype=torch.float32)
training_mom_z, val_mom_z, test_mom_z = torch.tensor(np.nan_to_num(training_mom_z), dtype=torch.float32), torch.tensor(np.nan_to_num(val_mom_z), dtype=torch.float32), torch.tensor(np.nan_to_num(test_mom_z), dtype=torch.float32)

print(training_kp.shape, val_lbl.shape, test_lengths.shape)
print(training_inpscale.shape, training_outscale.shape)

torch.Size([307, 250, 52]) torch.Size([39, 250, 26]) torch.Size([38])
torch.Size([307, 2]) torch.Size([307])


Finally we define the batch_size and put the datasets in DataLoaders.

In [80]:
train_data = TensorDataset(training_kp, training_lbl, training_lengths,
                          training_inpscale, training_outscale,
                          training_mom_x, training_mom_y, training_mom_z)
val_data = TensorDataset(val_kp, val_lbl, val_lengths,
                        val_inpscale, val_outscale,
                        val_mom_x, val_mom_y, val_mom_z)
test_data = TensorDataset(test_kp, test_lbl, test_lengths,
                         test_inpscale, test_outscale,
                         test_mom_x, test_mom_y, test_mom_z)

batch_size = 32

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size, drop_last=True)

print(train_loader)

If we have a GPU available we set our device to GPU.

In [81]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


Let's print some examples to see whether it is loaded correctly or not.

In [82]:
dataiter = iter(train_loader)
sample_x, sample_y, sample_len, iscale, oscale, momx, momy, momz = dataiter.next()

print(sample_x.shape, sample_y.shape, sample_len.shape, iscale.shape, momz.shape)

torch.Size([32, 250, 52]) torch.Size([32, 250, 26]) torch.Size([32]) torch.Size([32, 2]) torch.Size([32])


## Model building
It is time to build the model for this approach. It will consist on a single/double layer LSTM followed by a Linear layer with output size the number of keypoints we want to estimate. I also define a method to initialize the hidden_state of the cell.

In [83]:
class LSTM_2D3D(nn.Module):
    
    def __init__(self, input_size, output_size, hidden_dim, n_layers, bidirectional, dropout=0.):
        super().__init__()
        # Save the model parameters
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.bi = bidirectional
        
        # Define the architecture
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True, bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim*(2 if self.bi else 1), output_size)
        )
        
    def forward(self, x, state, lengths):
        # Describe the forward step
        batch_size, seq_len = x.size(0), x.size(1) # We save the batch size and the (maximum) sequence length
        
        # Need to pack a tensor containing padded sequences of variable length
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths=lengths, batch_first=True, enforce_sorted=False)
        ht, hidden_state = self.lstm(packed, state) # ht will be a PackedSequence

        # Need to flatten and reshape the output to feed it to the Linear layer
        ht = ht.data.contiguous() # ht will be of shape [sum(lengths), hidden_dim]
        ot = self.fc(ht) # ot will be of shape [sum(lengths), ouput_size]

        l_ot = [ot[:int(length)] for length in lengths] # list of batch elements, each shape [lengths[i], output_size]
        packed_ot = nn.utils.rnn.pack_sequence(l_ot, enforce_sorted=False) # PackedSequence
        # Finally return to shape [batch_size, seq_len, output_size]
        ot, _ = nn.utils.rnn.pad_packed_sequence(packed_ot, batch_first=True, total_length=seq_len)
        
        return ot, hidden_state
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers*(2 if self.bi else 1), batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers*(2 if self.bi else 1), batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [84]:
# Define some model parameters
INPUT_SIZE = sample_x.size(2)
OUTPUT_SIZE = sample_y.size(2)
HIDDEN_DIM = 512
N_LAYERS = 2
BIDIRECTIONAL = False

# Instantiate the model
model = LSTM_2D3D(INPUT_SIZE, OUTPUT_SIZE, HIDDEN_DIM, N_LAYERS, BIDIRECTIONAL, dropout=0.)
model.to(device)
print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

LSTM_2D3D(
  (lstm): LSTM(52, 512, num_layers=2, batch_first=True)
  (fc): Sequential(
    (0): Linear(in_features=512, out_features=26, bias=True)
  )
)
3273754


## Training
Now we will proceed with the training. The first cell will define the learning rate, the loss function and the selected optimizer for the training process. Then we will proceed with a training over a number of epochs in which we will print it's training loss and validation loss. I also will be using Tensorboard to have a much nicer view of the results.

In [85]:
def substract_root_PJPE(output):
    jb = torch.chunk(output, max_seq, dim=1)
    root = 8
    n_joints = []
    for chunk in jb:
        n_joints.append(chunk.sub(chunk[:,:,root].unsqueeze(2)))
    joints_merged = torch.cat(tuple(n_joints), dim=1)
    return joints_merged

In [86]:
def mpjpe(rooted_o, rooted_l, seq_lens):
    MPJPE = []
    for i in range(len(seq_lens)):
        MPJPE.append(rooted_o[i,:int(seq_lens[i])].sub(rooted_l[i,:int(seq_lens[i])]).abs().mean().item())
    
    return np.mean(MPJPE)
    

In [87]:
def correct_keypoints(output, labels, seq_lens, alpha=0.1):
    total_correct = 0
    for i in range(len(seq_lens)):
        for j in range(int(seq_lens[i])):
            rang = (labels[i,j].max()-labels[i,j].min())
            for o, l in zip(output[i,j], labels[i,j]):
                if o < l+alpha*rang and o > l-alpha*rang:
                    total_correct += 1
    return total_correct

In [88]:
NUM_EPOCHS = 35
lr = 6e-4
loss_function = nn.MSELoss()
one_cycle = True
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.)
if one_cycle:
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr, 
                                                  steps_per_epoch=len(train_loader), epochs=NUM_EPOCHS,
                                                  div_factor=25.0, final_div_factor=10000.0)

In [89]:
from datetime import datetime
name = 'body_whole'
writer = SummaryWriter(log_dir=f'/deeplearning/logs/{name}{datetime.now()}_lr-{lr}')

In [90]:
timer_beg = timer()

tr_losses = []
val_losses = []

model.train()
for i in range(NUM_EPOCHS):
    # Init the hidden state (ht, ct)
    h = model.init_hidden(batch_size)
    batch_losses = []
    train_MPJPE = []
    
    if i+1 == NUM_EPOCHS:
        preds, inps, labls, lens = [], [], [], []
        val_preds, val_inps, val_labls, val_lens = [], [], [], []
        iscale, oscale, val_iscale, val_oscale = [], [], [], []
        momx, momy, momz = [], [], []
        val_momx, val_momy, val_momz = [], [], []
        
    for inputs, labels, lengths, i_s, o_s, mx, my, mz in train_loader:
        h = tuple([e.data for e in h])
        inputs, labels, lengths = inputs.to(device), labels.to(device), lengths.to(device)
        # Clear the gradients
        optimizer.zero_grad()
        # Forward step
        output, h = model(inputs, h, lengths)
        
        if i+1 == NUM_EPOCHS:
            e = [preds, inps, labls, lens, iscale, oscale, momx, momy, momz]
            b = [output, inputs, labels, lengths, i_s, o_s, mx, my, mz]
            for k in range(len(e)):
                e[k].append(b[k])            

        # Loss calculation and backward step
        loss = loss_function(nn.utils.rnn.pack_padded_sequence(output, lengths=lengths, batch_first=True, 
                                                               enforce_sorted=False).data,
                             nn.utils.rnn.pack_padded_sequence(labels, lengths=lengths, batch_first=True,
                                                               enforce_sorted=False).data)
        loss.backward()
        # Weight update
        optimizer.step()
        # One cycle policy step
        if one_cycle:
            scheduler.step()
        
        # Output data collection for showing
        batch_losses.append(loss.item())
        rooted_o, rooted_l = substract_root_PJPE(output), substract_root_PJPE(labels)
        train_MPJPE.append(mpjpe(rooted_o, rooted_l, lengths))
    
    timer_end = timer()
    tr_losses.append(np.mean(batch_losses))
    writer.add_scalar('Loss/train', tr_losses[-1], i)   
    train_MPJPE_total = np.mean(train_MPJPE)
    
    # Validation at the end of an epoch
    val_h = model.init_hidden(batch_size)
    model.eval()
    val_loss = []
    val_MPJPE = []
    
    for inp, lab, lns, vis, vos, vmx, vmy, vmz in val_loader:
        val_h = tuple([each.data for each in val_h])
        inp, lab, lns = inp.to(device), lab.to(device), lns.to(device)
        out, val_h = model(inp, val_h, lns)
        
        if i+1 == NUM_EPOCHS:
            e = [val_preds, val_inps, val_labls, val_lens, val_iscale, val_oscale, val_momx, val_momy, val_momz]
            b = [out, inp, lab, lns, vis, vos, vmx, vmy, vmz]
            for k in range(len(e)):
                e[k].append(b[k])  
        
        loss = loss_function(nn.utils.rnn.pack_padded_sequence(out, lengths=lns, batch_first=True,
                                                               enforce_sorted=False).data,
                             nn.utils.rnn.pack_padded_sequence(lab, lengths=lns, batch_first=True,
                                                               enforce_sorted=False).data)
        val_loss.append(loss.item())
        rooted_o, rooted_l = substract_root_PJPE(out), substract_root_PJPE(lab)
        val_MPJPE.append(mpjpe(rooted_o, rooted_l, lns))
        
    val_losses.append(np.mean(val_loss))
    writer.add_scalar('Loss/validation', val_losses[-1], i)  
    val_MPJPE_total = np.mean(val_MPJPE)
    model.train()
    
    # Output loss and training time.
    print(f"Finished epoch {i+1}/{NUM_EPOCHS} in {(timer_end-timer_beg):.2f}s.\n",
             f"Loss: {np.mean(tr_losses[-1]):.4f}", f" Val Loss: {val_losses[-1]:.4f}\n",
             f"Train MPJPE: {train_MPJPE_total:.4f}", f" Val MPJPE: {val_MPJPE_total:.4f}")
    timer_beg = timer()

plt.figure()
plt.plot(tr_losses, label='train')
plt.plot(val_losses, label='validation')
plt.xlabel('Epoch')
plt.ylabel('MSELoss')
plt.legend()

Finished epoch 1/35 in 2.41s.
 Loss: 1.8263  Val Loss: 0.8813
 Train MPJPE: 1.2909  Val MPJPE: 0.9014
Finished epoch 2/35 in 2.38s.
 Loss: 1.7500  Val Loss: 0.8647
 Train MPJPE: 1.2551  Val MPJPE: 0.8820
Finished epoch 3/35 in 2.38s.
 Loss: 1.5248  Val Loss: 0.6615
 Train MPJPE: 1.1201  Val MPJPE: 0.7298
Finished epoch 4/35 in 2.38s.
 Loss: 0.8540  Val Loss: 0.4714
 Train MPJPE: 0.6684  Val MPJPE: 0.5323
Finished epoch 5/35 in 2.39s.
 Loss: 0.6873  Val Loss: 0.4106
 Train MPJPE: 0.5271  Val MPJPE: 0.4513
Finished epoch 6/35 in 2.40s.
 Loss: 0.6896  Val Loss: 0.4153
 Train MPJPE: 0.5089  Val MPJPE: 0.4663
Finished epoch 7/35 in 2.39s.
 Loss: 0.6523  Val Loss: 0.3958
 Train MPJPE: 0.4887  Val MPJPE: 0.4607
Finished epoch 8/35 in 2.39s.
 Loss: 0.6480  Val Loss: 0.3970
 Train MPJPE: 0.4783  Val MPJPE: 0.4531
Finished epoch 9/35 in 2.40s.
 Loss: 0.6455  Val Loss: 0.3868
 Train MPJPE: 0.4718  Val MPJPE: 0.4576
Finished epoch 10/35 in 2.39s.
 Loss: 0.6424  Val Loss: 0.3867
 Train MPJPE: 0.476

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Save the predictions for training and validation.

In [91]:
tr_predictions = torch.cat(tuple(preds), dim=0)
tr_inputs = torch.cat(tuple(inps), dim=0)
tr_groundtruth = torch.cat(tuple(labls), dim=0)
tr_lengths = torch.cat(tuple(lens), dim=0)
tr_inp_scale, tr_out_scale = torch.cat(tuple(iscale), dim=0), torch.cat(tuple(oscale), dim=0)
tr_mx, tr_my, tr_mz = torch.cat(tuple(momx), dim=0), torch.cat(tuple(momy), dim=0), torch.cat(tuple(momz), dim=0)
print(tr_inp_scale.shape, tr_inputs.shape)

val_predictions = torch.cat(tuple(val_preds), dim=0)
val_inputs = torch.cat(tuple(val_inps), dim=0)
val_groundtruth = torch.cat(tuple(val_labls), dim=0)
val_length = torch.cat(tuple(val_lens), dim=0)
val_inp_scale, val_out_scale = torch.cat(tuple(val_iscale), dim=0), torch.cat(tuple(val_oscale), dim=0)
val_mx, val_my, val_mz = torch.cat(tuple(val_momx), dim=0), torch.cat(tuple(val_momy), dim=0), torch.cat(tuple(val_momz), dim=0)

torch.Size([288, 2]) torch.Size([288, 250, 52])


In [92]:
c_output = tr_predictions.clone()
c_labels = tr_groundtruth.clone()
for vid in range(c_labels.shape[0]): 
    c_output[vid].mul_(tr_mz[vid])
    c_labels[vid].mul_(tr_mz[vid])

tr_CK = correct_keypoints(c_output, c_labels, tr_lengths)
tr_K = tr_lengths.sum()*tr_groundtruth.shape[2]

c_output = val_predictions.clone()
c_labels = val_groundtruth.clone()
for vid in range(c_labels.shape[0]): 
    c_output[vid].mul_(val_mz[vid,0])
    c_labels[vid].mul_(val_mz[vid,1])
    
val_CK = correct_keypoints(c_output, c_labels, val_length)
val_K = val_lengths.sum()*val_groundtruth.shape[2]

tr_PCK = tr_CK/ tr_K
val_PCK = val_CK/ val_K
print(f"Training PCK: {tr_PCK*100:.3f} Validation PCK: {val_PCK*100:.3f}")

Training PCK: 52.223 Validation PCK: 36.455


In [ ]:
zeros = torch.zeros((138), device='cuda:0')

for i in range(len(tr_predictions)):
    count=0
    for frame in tr_predictions[i, :int(tr_lengths[i])]:
        if torch.all(frame.eq(zeros)):
            count +=1
    print(f'{count}/{int(tr_lengths[i])}')

In [ ]:
zeros = torch.zeros((138), device='cuda:0')

for i in range(len(val_predictions)):
    count=0
    for frame in tr_predictions[i, :int(val_length[i])]:
        if torch.all(frame.eq(zeros)):
            count +=1
    print(f'{count}/{int(val_length[i])}')

In [93]:
torch.save(model.state_dict(), f'./{name}.pt')

In [94]:
model.load_state_dict(torch.load(f'./{name}.pt'))

<All keys matched successfully>

## Testing
After the training, we shall proceed with the performance test. This will go through the test batches and perform the inference, then it will show the test loss, as well as the performance metric. In this case, as we are working with human body keypoints, we will use the Mean Per Joint Position Error (MPJPE) metric, which outputs the mean euclidean distance between the joints (keypoints) positions estimated and the ones in the groundtruth.

The formula for MPJPE is the following:

$\text{MPJPE} = \frac1T\frac1N\displaystyle\sum_{t=1}^{T}\displaystyle\sum_{i=1}^{N}\|(J_{i}^{(t)}-J_{root}^{(t)})-(Ĵ_{i}^{(t)}-Ĵ_{root}^{(t)})\|$

Where N is the number of joints, and T the number of samples.

As we have seen in the formula above, we need to align the root joints of the labels and the network output. In order to do that, I have defined a function (`substract_root_PJPE`) that substracts the root joint of each keypoint set (face, hands, body) in the corresponding keypoint set.

In [95]:
test_losses = []
MPJPE = []
h = model.init_hidden(batch_size)
preds, inps, labls, lengs = [], [], [], []
iscal, oscal = [], []
mx, my, mz = [], [], []

model.eval()
for inputs_test, labels_test, lengths_test, is_test, os_test, mx_test, my_test, mz_test in test_loader:
    
    h = tuple([each.data for each in h])
    inputs_test, labels_test, lengths_test = inputs_test.to(device), labels_test.to(device), lengths_test.to(device)
    
    output_test, h = model(inputs_test, h, lengths_test)
    
    e = [preds, inps, labls, lengs, iscal, oscal, mx, my, mz]
    b = [output_test, inputs_test, labels_test, lengths_test, is_test, os_test, mx_test, my_test, mz_test]
    for k in range(len(e)):
        e[k].append(b[k])      
    
    test_loss = loss_function(nn.utils.rnn.pack_padded_sequence(output_test, lengths=lengths_test, batch_first=True,
                                                               enforce_sorted=False).data,
                             nn.utils.rnn.pack_padded_sequence(labels_test, lengths=lengths_test, batch_first=True,
                                                               enforce_sorted=False).data)
    test_losses.append(test_loss.item())
    rooted_o, rooted_l = substract_root_PJPE(output_test), substract_root_PJPE(labels_test)
    MPJPE.append(mpjpe(rooted_o, rooted_l, lengths_test))


test_predictions = torch.cat(tuple(preds), dim=0)
test_inputs = torch.cat(tuple(inps), dim=0)
test_groundtruth = torch.cat(tuple(labls), dim=0)
test_lengths = torch.cat(tuple(lengs), dim=0)
test_inp_scale, test_out_scale = torch.cat(tuple(iscal), dim=0), torch.cat(tuple(oscal), dim=0)
test_mx, test_my, test_mz = torch.cat(tuple(mx), dim=0), torch.cat(tuple(my), dim=0), torch.cat(tuple(mz), dim=0)
MPJPE_total = np.mean(MPJPE)

c_output = test_predictions.clone()
c_labels = test_groundtruth.clone()
for vid in range(c_labels.shape[0]): 
    c_output[vid].mul_(test_mz[vid,0])
    c_labels[vid].mul_(test_mz[vid,1])
    
test_CK = correct_keypoints(c_output, c_labels, test_lengths)
test_K = test_lengths.sum()*test_groundtruth.shape[2]

test_PCK = test_CK/ test_K

In [96]:
print(f"Test loss: {np.mean(test_losses):.4f}", f"\nMPJPE: {MPJPE_total:.4f}",  f"\nTest PCK: {test_PCK*100:.3f}")

Test loss: 0.4029 
MPJPE: 0.4508 
Test PCK: 44.011


### Save the results into a json

In [2]:
results = {'train':{'inputs':tr_inputs.tolist(), 'predictions':tr_predictions.tolist(), 'labels':tr_groundtruth.tolist(), 'lengths':tr_lengths.tolist(), 
                    'is':tr_inp_scale.tolist(), 'os':tr_out_scale.tolist(), 'mx':tr_mx.tolist(), 'my':tr_my.tolist(), 'mz':tr_mz.tolist()},
          'validation':{'inputs':val_inputs.tolist(), 'predictions':val_predictions.tolist(), 'labels':val_groundtruth.tolist(), 'lengths':val_length.tolist(),
                       'is':val_inp_scale.tolist(), 'os':val_out_scale.tolist(), 'mx':val_mx.tolist(), 'my':val_my.tolist(), 'mz':val_mz.tolist()},
          'test':{'inputs':test_inputs.tolist(), 'predictions':test_predictions.tolist(), 'labels':test_groundtruth.tolist(), 'lengths':test_lengths.tolist(),
                 'is':test_inp_scale.tolist(), 'os':test_out_scale.tolist(), 'mx':test_mx.tolist(), 'my':test_my.tolist(), 'mz':test_mz.tolist()}}
with open('../../results/small_body.json', 'w') as fp:
    json.dump(results, fp)

NameError: name 'tr_inputs' is not defined

### Load results from json

In [106]:
with open('../../results/small_body_A.json', 'r') as j:
    jd = json.load(j)
    tr, val, test = jd['train'], jd['validation'], jd['test']
    tr_inputs, tr_predictions, tr_groundtruth, tr_lengths, tr_inp_scale, tr_out_scale, tr_mx, tr_my, tr_mz = tuple(torch.tensor(tr[n]) for n in ['inputs', 'predictions',
                                                                                              'labels', 'lengths',
                                                                                               'is', 'os', 'mx', 'my', 'mz'])
    val_inputs, val_predictions, val_groundtruth, val_length, val_inp_scale, val_out_scale, val_mx, val_my, val_mz = tuple(torch.tensor(val[n]) for n in ['inputs', 'predictions',
                                                                                                  'labels', 'lengths',
                                                                                                  'is', 'os', 'mx', 'my', 'mz'])
    test_inputs, test_predictions, test_groundtruth, test_lengths, test_inp_scale, test_out_scale, test_mx, test_my, test_mz = tuple(torch.tensor(test[n]) for n in ['inputs', 'predictions',
                                                                                                         'labels', 'lengths',
                                                                                                         'is', 'os', 'mx', 'my', 'mz'])

In [107]:
tr_inputs.shape, val_predictions.shape, test_lengths.shape

(torch.Size([768, 350, 52]), torch.Size([96, 350, 26]), torch.Size([96]))

## Interpreation

Now to better understanding of the results, I will plot some of the frames from the last batches on the training and validation, and also from testing.

In [108]:
def plot_and_rotate(c_inputs, c_z, frames, frame):
    c_inputs[:,::2].mul_(mom_x[1])
    c_inputs[:,1::2].mul_(mom_y[1])
    c_z.mul_(stdz)

    bodiesXY = torch.chunk(c_inputs[frames, :], len(frames), dim=0)
    bodiesZ = torch.chunk(c_z[frames, :], len(frames), dim=0)
    
    x = bodiesXY[frame].squeeze()[::2]
    y = bodiesXY[frame].squeeze()[1::2]
    z = bodiesZ[frame].squeeze()

    l_arm = [[c[i] for i in [1, 0, 9, 10, 11]] for c in [x,y,z]]
    r_arm = [[c[i] for i in [0, 3, 4, 5]] for c in [x,y,z]]
    l_leg = [[c[i] for i in [0, 2, 12, 13, 14, 22, 23, 24]] for c in [x,y,z]]
    r_leg = [[c[i] for i in [2, 6, 7, 8, 19, 20, 21]] for c in [x,y,z]]
    head = [[c[i] for i in [18, 17, 1, 15, 16]] for c in [x,y,z]]

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    def init():
        ax.plot(r_arm[0], r_arm[2], r_arm[1])
        ax.plot(l_arm[0], l_arm[2], l_arm[1])
        ax.plot(r_leg[0], r_leg[2], r_leg[1])
        ax.plot(l_leg[0], l_leg[2], l_leg[1])
        ax.plot(head[0], head[2], head[1])
        
        lims = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()
        spans = lims[0][1]-lims[0][0], lims[1][1]-lims[1][0], lims[2][1]-lims[2][0]
        span = max(spans)
        margins = [(span-s)/2 for  s in spans]
        ax.set_xlim(lims[0][0]-margins[0], lims[0][1]+margins[0])
        ax.set_ylim(lims[1][0]-margins[1], lims[1][1]+margins[1])
        ax.set_zlim(lims[2][0]-margins[2], lims[2][1]+margins[2])
        
        return fig,

    def animate(i):
        ax.view_init(elev=220., azim=3.6*i)
        return fig,

    # Animate
    ani = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=100, blit=True)    

    return ani

In [109]:
def plot_frames(predictions, groundtruth, inputs, video_n, frames, rot, train=False):
    
    inp = inputs.clone()
    preds = predictions.clone()
    grtr = groundtruth.clone()

    bodiesXY = torch.chunk(inp[video_n, frames, :], len(frames), dim=0)
    pred_bodiesZ = torch.chunk(preds[video_n, frames, :], len(frames), dim=0)
    true_bodiesZ = torch.chunk(grtr[video_n, frames, :], len(frames), dim=0)
   
    nrows = np.ceil(len(frames)/2)
    fig = plt.figure(figsize=(15, 6*nrows))
    fig2 = plt.figure(figsize=(15, 6*nrows))
    for frame in range(len(frames)):
        x = bodiesXY[frame].squeeze()[::2].tolist()
        y = bodiesXY[frame].squeeze()[1::2].tolist()
        pred_z = pred_bodiesZ[frame].squeeze().tolist()
        true_z = true_bodiesZ[frame].squeeze().tolist()
        
        r = R.from_euler('y', rot, degrees=True)
        
        xyz1, xyz2 = np.asarray([c for c in zip(x, y, pred_z)]), np.asarray([c for c in zip(x, y, true_z)])
        xyz1, xyz2 = r.apply(xyz1), r.apply(xyz2)
        x1, x2 = xyz1[:,0], xyz2[:,0]
        y1, y2 = xyz1[:,1], xyz2[:,1]
        pred_z, true_z = xyz1[:,2], xyz2[:,2]
        
        if not train:
            print((x2.max()-x2.min())/(x1.max()-x1.min()))
            x1 = x1*((x2.max()-x2.min())/(x1.max()-x1.min()))
        
        r_arm = tuple([[c[i] for i in [1, 0, 9, 10, 11]] for c in l] for l in [[x1, y1, pred_z], [x2, y2, true_z]])
        l_arm = tuple([[c[i] for i in [0, 3, 4, 5]] for c in l] for l in [[x1, y1, pred_z], [x2, y2, true_z]])
        r_leg = tuple([[c[i] for i in [0, 2, 12, 13, 14, 22, 23, 24]] for c in l] for l in [[x1, y1, pred_z], [x2, y2, true_z]])
        l_leg = tuple([[c[i] for i in [2, 6, 7, 8, 19, 20, 21]] for c in l] for l in [[x1, y1, pred_z], [x2, y2, true_z]])
        head = tuple([[c[i] for i in [18, 17, 1, 15, 16]] for c in l] for l in [[x1, y1, pred_z], [x2, y2, true_z]])

        ax = fig.add_subplot(nrows, 2, frame+1, projection='3d')

        ax.plot(r_arm[0][0], r_arm[0][1], r_arm[0][2])
        ax.plot(l_arm[0][0], l_arm[0][1], l_arm[0][2])
        ax.plot(r_leg[0][0], r_leg[0][1], r_leg[0][2])
        ax.plot(l_leg[0][0], l_leg[0][1], l_leg[0][2])
        ax.plot(head[0][0], head[0][1], head[0][2])
        
        ax2 = fig2.add_subplot(nrows, 2, frame+1, projection='3d')
        ax2.plot(r_arm[1][0], r_arm[1][1], r_arm[1][2])
        ax2.plot(l_arm[1][0], l_arm[1][1], l_arm[1][2])
        ax2.plot(r_leg[1][0], r_leg[1][1], r_leg[1][2])
        ax2.plot(l_leg[1][0], l_leg[1][1], l_leg[1][2])
        ax2.plot(head[1][0], head[1][1], head[1][2])
        
        lims = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()
        spans = lims[0][1]-lims[0][0], lims[1][1]-lims[1][0], lims[2][1]-lims[2][0]
        span = max(spans)
        margins = [(span-s)/2 for  s in spans]
        ax.set_xlim(lims[0][0]-margins[0], lims[0][1]+margins[0])
        ax.set_ylim(lims[1][0]-margins[1], lims[1][1]+margins[1])
        ax.set_zlim(lims[2][0]-margins[2], lims[2][1]+margins[2])
        
        lims2 = ax2.get_xlim(), ax2.get_ylim(), ax2.get_zlim()
        spans2 = lims2[0][1]-lims2[0][0], lims2[1][1]-lims2[1][0], lims2[2][1]-lims2[2][0]
        span2 = max(spans2)
        margins2 = [(span2-s)/2 for  s in spans2]
        ax2.set_xlim(lims2[0][0]-margins2[0], lims2[0][1]+margins2[0])
        ax2.set_ylim(lims2[1][0]-margins2[1], lims2[1][1]+margins2[1])
        ax2.set_zlim(lims2[2][0]-margins2[2], lims2[2][1]+margins2[2])

        ax.view_init(elev=-65., azim=-90.)
        ax2.view_init(elev=-65., azim=-90.)


### Single frame
On the first cell you can select which frames you want to plot and from which video of the batch. On the second you select which frame of the previosly selected you want to plot, specifying its index on the declared "frames" list.

In [ ]:
# Last batches of training -output, inputs, labels-.
vid = 1
frames = [100]

c_inputs = training_kp[vid].clone()
c_labels = training_lbl[vid].clone()
c_inputs[:,::2].mul_(training_inpscale[vid, 0])
c_inputs[:,1::2].mul_(training_inpscale[vid, 1])
c_labels.mul_(training_outscale[vid])

In [ ]:
HTML(plot_and_rotate(c_inputs, c_output, frames, 0).to_html5_video())

In [ ]:
c_inputs = tr_inputs[vid].clone()
HTML(plot_and_rotate(c_inputs, c_labels, frames, 0).to_html5_video())

### Slice of frames
Now let's plot a sequence of frames of the selected video. We will plot both the groundtruth and the predicted.

In [74]:
frames = [i for i in range(1,9,2)]
video_n = 34

c_inputs = tr_inputs.clone()
c_output = tr_predictions.clone()
c_labels = tr_groundtruth.clone()
print(c_inputs.dtype, c_output.dtype, c_labels.dtype)
for vid in range(c_labels.shape[0]): 
    c_inputs[vid,:,::2].mul_(tr_inp_scale[vid, 0])
    c_inputs[vid,:,1::2].mul_(tr_inp_scale[vid, 1])
    c_output[vid].mul_(tr_out_scale[vid])
    c_labels[vid].mul_(tr_out_scale[vid])
    
    c_inputs[vid,:,::2].mul_(tr_mx[vid, 1])
    c_inputs[vid,:,1::2].mul_(tr_my[vid, 1])
    c_output[vid].mul_(tr_mz[vid])
    c_labels[vid].mul_(tr_mz[vid])


torch.float32 torch.float32 torch.float32


In [75]:
plot_frames(c_output, c_labels, c_inputs, video_n, frames, -90, True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
frames = [i for i in range(100,180,20)]
video_n = 31

c_inputs = test_inputs.clone()
c_output = test_predictions.clone()
c_labels = test_groundtruth.clone()

for vid in range(c_labels.shape[0]): 
    c_inputs[vid,:,::2].mul_(test_inp_scale[vid, 0])
    c_inputs[vid,:,1::2].mul_(test_inp_scale[vid, 1])
    c_output[vid].mul_(test_out_scale[vid,0])
    c_labels[vid].mul_(test_out_scale[vid,1])
    
    c_inputs[vid,:,::2].mul_(test_mx[vid, 1])
    c_inputs[:,:,1::2].mul_(test_my[vid, 1])
    c_output[vid].mul_(test_mz[vid,0])
    c_labels[vid].mul_(test_mz[vid,1])


In [111]:
plot_frames(c_output, c_labels, c_inputs, video_n, frames, -90)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

5.794992463138675
4.401840232295766
4.3023408497738584
4.110155115829824


In [95]:
plt.close('all')

In [15]:
frames = [i for i in range(1,9,2)]
video_n = 151

c_inputs = torch.tensor(dataset)
c_output = torch.tensor(groundtruth)
c_labels = torch.tensor(groundtruth)


In [16]:
plot_frames(c_output, c_labels, c_inputs, video_n, frames, -90)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.0
1.0
1.0
1.0


In [56]:
frames = [i for i in range(1,9,2)]
video_n = 17

c_inputs = val_inputs.clone()
c_output = val_predictions.clone()
c_labels = val_groundtruth.clone()

for vid in range(c_labels.shape[0]): 
    c_inputs[vid,:,::2].mul_(val_inp_scale[vid, 0])
    c_inputs[vid,:,1::2].mul_(val_inp_scale[vid, 1])
    c_output[vid].mul_(val_out_scale[vid])
    c_labels[vid].mul_(val_out_scale[vid])
    
    c_inputs[vid,:,::2].mul_(val_mom_x[vid, 1])
    c_inputs[:,:,1::2].mul_(val_mom_y[vid, 1])
    c_output[vid].mul_(val_mom_z[vid])
    c_labels[vid].mul_(val_mom_z[vid])


In [57]:
plot_frames(c_output, c_labels, c_inputs, video_n, frames, -90)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …